# Class Weight Compution

In [7]:
import math
import sys

import matplotlib.pyplot as plt
import numpy as np
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
export_ds = '/mnt/data/datasets/nuscenes/processed'
# export_ds = '/media/scratch/berlukas/nuscenes'

sem_clouds_filename = f"{export_ds}/sem_bkp/sem_classes_gt1.npy"

print(f"Loading sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename).astype(int)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading sem clouds from /mnt/data/datasets/nuscenes/processed/sem_bkp/sem_classes_gt1.npy
Shape of sem clouds is (11230, 200, 200)


In [9]:
n_classes = 9
class_counts = [0] * n_classes
for cl in range(0, n_classes): 
    print(f'Computing occurrences for {cl}')
    class_counts[cl] = np.count_nonzero(sem_cloud_features[:,:,:] == cl)    

Computing occurrences for 0
Computing occurrences for 1
Computing occurrences for 2
Computing occurrences for 3
Computing occurrences for 4
Computing occurrences for 5
Computing occurrences for 6
Computing occurrences for 7
Computing occurrences for 8


In [5]:
print(f'Counts are \n{class_counts}')
total_counts = np.sum(class_counts)
rates = class_counts / total_counts
print(f'Should sum to one: {np.sum(rates) == 1.0}')
print(f'\nRates are\n====================\n')
print(f'Background: {rates[0]}')
print(f'Vehicle: {rates[1]}')
print(f'Person: {rates[2]}')
print(f'Objects: {rates[3]}')
print(f'Ground: {rates[4]}')
print(f'Buildings: {rates[5]}')
print(f'Vegetation: {rates[6]}')
print(f'Other: {rates[7]}')
print(f'Ego: {rates[8]}')

inv_rates = 1-rates
# inv_rates[0] = 0
# inv_rates[8] = 0
weights = np.round_((inv_rates) / np.sum(inv_rates), 20)

print(f'\nWeights are\n====================\n')
print(f'Background: {weights[0]}')
print(f'Vehicle: {weights[1]}')
print(f'Person: {weights[2]}')
print(f'Objects: {weights[3]}')
print(f'Ground: {weights[4]}')
print(f'Buildings: {weights[5]}')
print(f'Vegetation: {weights[6]}')
print(f'Other: {weights[7]}')
print(f'Ego: {weights[8]}')

print(f'\n ================= \n')
print(f'Weights sum to one: {(np.sum(weights) - 1.0) < 1e-3} ({np.sum(weights)})')

print(f'weights: {weights}')

Counts are 
[333913860, 1658322, 5022673, 2507157, 47208762, 20522035, 51577, 15313117, 23002497]
Should sum to one: True

Rates are

Background: 0.7433523152270703
Vehicle: 0.0036917230632235085
Person: 0.01118137355298308
Objects: 0.005581382457702583
Ground: 0.10509519590382903
Buildings: 0.04568574131789849
Vegetation: 0.00011481967943009796
Other: 0.03408975289403384
Ego: 0.05120769590382903

Weights are

Background: 0.032080960596616206
Vehicle: 0.12453853461709705
Person: 0.12360232830587711
Objects: 0.12430232719278717
Ground: 0.11186310051202138
Buildings: 0.11928928233526268
Vegetation: 0.12498564754007124
Other: 0.12073878088824579
Ego: 0.11859903801202137


Weights sum to one: True (1.0)
weights: [0.03208096 0.12453853 0.12360233 0.12430233 0.1118631  0.11928928
 0.12498565 0.12073878 0.11859904]


In [11]:
print(f'Counts are \n{class_counts}')
total_counts = np.sum(class_counts)

weights = np.zeros(n_classes)
for i in range(0,n_classes):
    weights[i] = total_counts / (n_classes * class_counts[i])

# weights[0] = 0
# weights[8] = 0
weights /= np.sum(weights)
    
print(f'\nWeights are\n====================\n')
print(f'Background: {weights[0]}')
print(f'Vehicle: {weights[1]}')
print(f'Person: {weights[2]}')
print(f'Objects: {weights[3]}')
print(f'Ground: {weights[4]}')
print(f'Buildings: {weights[5]}')
print(f'Vegetation: {weights[6]}')
print(f'Other: {weights[7]}')
print(f'Ego: {weights[8]}')

print(f'\n ================= \n')
print(f'Weights sum to one: {(np.sum(weights) - 1.0) < 1e-3} ({np.sum(weights)})')

Counts are 
[1299164, 6344477, 2690889, 44632404, 23522282, 153156, 355570704, 23866924]

Weights are

Vehicle: 0.09696859097381993
Person: 0.019856341590317347
Objects: 0.046816536291133456
Ground: 0.002822570402524404
Buildings: 0.005355692212341975
Vegetation: 0.8225476150063452
Other: 0.0003542983184686436
Ego: 0.005278355205049122


Weights sum to one: True (1.0)


In [10]:
print(f'Counts are \n{class_counts}')
max_class = np.amax(class_counts)
print(f'Maximum is {max_class}')

weights = max_class / class_counts


print(f'\nWeights are\n====================\n')
print(f'Background: {weights[0]}')
print(f'Vehicle: {weights[1]}')
print(f'Person: {weights[2]}')
print(f'Objects: {weights[3]}')
print(f'Ground: {weights[4]}')
print(f'Buildings: {weights[5]}')
print(f'Vegetation: {weights[6]}')
print(f'Other: {weights[7]}')
print(f'Ego: {weights[8]}')

print(f'\n ================= \n')
print(f'Weights sum to one: {(np.sum(weights) - 1.0) < 1e-3} ({np.sum(weights)})')

print(f'weights: {np.round(weights,2)}')

Counts are 
[334092399, 1658322, 5022673, 2507157, 47208762, 20522035, 51577, 15134578, 23002497]
Maximum is 334092399

Weights are

Background: 1.0
Vehicle: 201.4641300061146
Person: 66.51685248074084
Objects: 133.2554758238116
Ground: 7.076915065046611
Buildings: 16.279691512074702
Vegetation: 6477.546173682068
Other: 22.074774665008828
Ego: 14.524179657538918


Weights sum to one: False (6939.738192892404)
weights: [1.00000e+00 2.01460e+02 6.65200e+01 1.33260e+02 7.08000e+00 1.62800e+01
 6.47755e+03 2.20700e+01 1.45200e+01]


In [15]:
np.amax(np.array([1,2,3,4,5]))

5